# TerraClassify: Deep Learning-based Land Cover Classification and Semantic Segmentation for Sustainable Development and Automation

### Contributors:

#### Dharu Piraba Muguntharaman (dm5596@nyu.edu), Raunak Bhupal (rb4986@nyu.edu), Shamyukta Rajagopal (sr6626@nyu.edu)

We have selected this topic to work on in accordance with the Final Project for CS-GY 6953 Deep Learning for the Academic Semester Spring 2023.

Implementation has been provided for the code to run on Colab

We uploaded the data on Google Drive and then mount this drive on Google Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip /content/drive/MyDrive/data/data.zip

Importing required libraries

In [ ]:
import numpy as np
from PIL import Image
from sklearn.preprocessing import MinMaxScaler
from patchify import patchify

minmaxscaler = MinMaxScaler()

Let's load the dataset first

There are 683 satellite and it's corresponding mask images in the training_data folder, but due to compute restrictions we were unable to load the entire dataset and thus just took the first 300 satellite and it's corresponding mask images

In [ ]:
import os
import cv2

# Path to the directory containing images
image_data_path = '/content/data/training_data/images/'
mask_data_path = '/content/data/training_data/masks/'

count = 0
image_patch_size = 256
image_dataset = []
mask_dataset = []

files = sorted(os.listdir(image_data_path))

#Iterate over files in the directory
for filename in files:
    if count == 300:
        break
    count+=1
    if filename.endswith('.jpg'):
        file_path = os.path.join(image_data_path, filename)
        image = cv2.imread(file_path)
        if image is not None:
            #image shape is (height,width)
            width = (image.shape[1]//image_patch_size)*image_patch_size
            height = (image.shape[0]//image_patch_size)*image_patch_size
            #convert to PIL
            image = Image.fromarray(image)
            #crop is (left, top, right, bottom)
            image = image.crop((0,0, width, height))
            image = np.array(image)
            #create patches of images
            patched_images = patchify(image,(image_patch_size, image_patch_size,3),step =image_patch_size)
            for i in range(patched_images.shape[0]):
                for j in range(patched_images.shape[1]): 
                    individual_image = patched_images[i,j,:,:]
                    individual_image  = minmaxscaler.fit_transform(individual_image.reshape(-1, individual_image.shape[-1])).reshape(individual_image.shape)
                    individual_image  = individual_image[0]
            image_dataset.append(image)

count = 0
files = sorted(os.listdir(mask_data_path))
#Iterate over files in the directory
for filename in files:
    if count == 300:
        break
    count+=1
    if filename.endswith('.png'):
        file_path = os.path.join(mask_data_path, filename)
        image = cv2.imread(file_path)
        if image is not None:
            #need to convert masks to RGB in accordance with the metadata
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            #image shape is (height,width)
            width = (image.shape[1]//image_patch_size)*image_patch_size
            height = (image.shape[0]//image_patch_size)*image_patch_size
            #convert to PIL
            image = Image.fromarray(image)
            #crop is (left, top, right, bottom)
            image = image.crop((0,0, width, height))
            image = np.array(image)
            #create patches of images
            patched_images = patchify(image,(image_patch_size, image_patch_size,3),step =image_patch_size)
            for i in range(patched_images.shape[0]):
                for j in range(patched_images.shape[1]): 
                    individual_image = patched_images[i,j,:,:]
                    individual_image  = individual_image[0]
            mask_dataset.append(image)

print("Success")

In [ ]:
image_dataset = np.array(image_dataset)
mask_dataset = np.array(mask_dataset)

In [ ]:
# Code to plot a random image and it's mask
import random
import matplotlib.pyplot as plt
random_image_id = random.randint(0, len(image_dataset))  # viewing random images from the dataset 
print(random_image_id)

#plotting the images 
plt.figure(figsize = (14,8))
plt.subplot(121)
plt.imshow(image_dataset[random_image_id])
plt.subplot(122)
plt.imshow(mask_dataset[random_image_id])

Creating the different classes. Values are assigned in accordance with the metadata description for this dataset, available on kaggle : https://www.kaggle.com/datasets/balraj98/deepglobe-land-cover-classification-dataset?resource=download

In [ ]:
class_urban_land = np.array([0,255,255])

class_agriculture_land = np.array([255,255,0])

class_rangeland = np.array([255,0,255])

class_forest_land = np.array([0,255,0])

class_water = np.array([0,0,255])

class_barren_land = np.array([255,255,255])

class_unknown = np.array([0,0,0])

In [ ]:
def rgb_to_label(label):
    label_segment = np.zeros(label.shape, dtype=np.uint8)
    label_segment[np.all(label == class_urban_land , axis=-1)] = 0
    label_segment[np.all(label == class_agriculture_land, axis=-1)] = 1
    label_segment[np.all(label == class_rangeland , axis=-1)] = 2
    label_segment[np.all(label == class_forest_land, axis=-1)] = 3
    label_segment[np.all(label == class_water, axis=-1)] = 4
    label_segment[np.all(label == class_barren_land, axis=-1)] = 5
    label_segment[np.all(label == class_unknown, axis=-1)] = 6
    label_segment = label_segment[:,:,0]
    return label_segment

#Creating labels
labels = []
for i in range(len(mask_dataset)):
    label = rgb_to_label(mask_dataset[i])
    labels.append(label)

In [ ]:
labels = np.array(labels)
labels = np.expand_dims(labels, axis=3)

In [ ]:
classes = np.unique(labels)
classes = len(classes)

In [ ]:
print("Total unique labels based on masks: ",format(classes))

In [ ]:
#Converting the labels to categorical dataset

from tensorflow.keras.utils import to_categorical 
labels_categorical_dataset = to_categorical(labels, num_classes=classes)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(image_dataset, labels_categorical_dataset, test_size =0.20, random_state = 42)

In [ ]:
!pip install -U -q segmentation-models
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ["SM_FRAMEWORK"] = "tf.keras"

from tensorflow import keras
import segmentation_models as sm

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose
from tensorflow.keras.layers import concatenate, BatchNormalization, Dropout, Lambda
from tensorflow.keras import backend as K

In [ ]:
#loss coefficient
def jaccard_coef(y_true, y_pred):
    y_true_flatten = K.flatten(y_true)
    y_pred_flatten = K.flatten(y_pred)
    intersection = K.sum(y_true_flatten * y_pred_flatten)
    final_coef_value = (intersection + 1.0) / (K.sum(y_true_flatten) + K.sum(y_pred_flatten) - intersection + 1.0)
    return final_coef_value

Let's define the Unet model 

In [ ]:
def unet_model(n_classes=classes, image_height=256, image_width=256, image_channels=1):

    inputs = Input((image_height, image_width, image_channels))
    
    c1 = Conv2D(32, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(inputs)
    c1 = Dropout(0.2)(c1)
    c1 = Conv2D(32, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(c1)
    p1 = MaxPooling2D((2,2))(c1)

    c2 = Conv2D(64, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(p1)
    c2 = Dropout(0.2)(c2)
    c2 = Conv2D(64, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(c2)
    p2 = MaxPooling2D((2,2))(c2)

    c3 = Conv2D(128, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(p2)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(128, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(c3)
    p3 = MaxPooling2D((2,2))(c3)

    c4 = Conv2D(256, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(p3)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(256, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(c4)
    p4 = MaxPooling2D((2,2))(c4)

    c5 = Conv2D(512, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(p4)
    c5 = Dropout(0.2)(c5)
    c5 = Conv2D(512, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(c5)


    u6 = Conv2DTranspose(256, (2,2), strides=(2,2), padding="same")(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(256, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(u6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(256, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(c6)

    u7 = Conv2DTranspose(128, (2,2), strides=(2,2), padding="same")(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(128, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(128, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(c7)

    u8 = Conv2DTranspose(64, (2,2), strides=(2,2), padding="same")(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(64, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(u8)
    c8 = Dropout(0.2)(c8)
    c8 = Conv2D(64, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(c8)

    u9 = Conv2DTranspose(32, (2,2), strides=(2,2), padding="same")(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(32, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(u9)
    c9 = Dropout(0.2)(c9)
    c9 = Conv2D(32, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(c9)

    outputs = Conv2D(n_classes, (1,1), activation="softmax")(c9)

    model = Model(inputs=[inputs], outputs=[outputs])
    return model

In [ ]:
metrics = ["accuracy", jaccard_coef]

In [ ]:
image_height = X_train.shape[1]
image_width = X_train.shape[2]
image_channels = X_train.shape[3]

model = unet_model(n_classes=classes, image_height=image_height, image_width=image_width, image_channels=image_channels)

Below we are defining the weights, by equally dividing the values among the 7 classes

In [ ]:
weights = [0.1428, 0.1428, 0.1428, 0.1428, 0.1428, 0.1428, 0.1428]

In [ ]:
dice_loss = sm.losses.DiceLoss(class_weights = weights)
focal_loss = sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss)

In [ ]:
optimizer = keras.optimizers.Adam(0.0001)
model.compile(optimizer=optimizer, loss=total_loss, metrics=metrics)

In [ ]:
model.summary()

In [ ]:
model_history = model.fit(X_train, y_train,batch_size=16,verbose=1,epochs=100,validation_data=(X_test, y_test),shuffle=False)

Let's visualize the plots

In [ ]:
fig, axs = plt.subplots(1, 2, figsize = (18, 5))

loss = model_history.history['loss']            
val_loss = model_history.history['val_loss']
epochs = range(1, len(loss) + 1)

axs[0].plot(epochs, loss, 'y', label="Training Loss")
axs[0].plot(epochs, val_loss, 'r', label="Validation Loss")
axs[0].set_title("My_Unet_Training Vs Validation Loss")
axs[0].set_xlabel("Epochs")
axs[0].set_ylabel("Loss")
axs[0].legend()

jaccard_coef = model_history.history['jaccard_coef']       
val_jaccard_coef = model_history.history['val_jaccard_coef']
epochs = range(1, len(jaccard_coef) + 1)

axs[1].plot(epochs, jaccard_coef, 'y', label="Training IoU")
axs[1].plot(epochs, val_jaccard_coef, 'r', label="Validation IoU")
axs[1].set_title("My_Unet_Training Vs Validation IoU")
axs[1].set_xlabel("Epochs")
axs[1].set_ylabel("Loss")
plt.legend()
plt.save('loss_iou.png')

In [ ]:
fig, axs = plt.subplots(1, 1, figsize = (18, 5))

accuracy = model_history.history['accuracy']             
val_accuracy = model_history.history['val_accuracy']     
epochs = range(1, len(loss) + 1)

axs[0].plot(epochs, accuracy, 'y', label="Training accuracy")
axs[0].plot(epochs, val_accuracy, 'r', label="Validation accuracy")
axs[0].set_title("My_Unet_Training Vs Validation accuracy")
axs[0].set_xlabel("Epochs")
axs[0].set_ylabel("accuracy")
axs[0].legend()
plt.save('accuracy_curve.png')

In [ ]:
y_pred = model.predict(X_test)
y_pred_argmax = np.argmax(y_pred, axis=3)
y_test_argmax = np.argmax(y_test, axis=3)

test_image_number = random.randint(0, len(X_test))

test_image = X_test[test_image_number]
ground_truth_image = y_test_argmax[test_image_number]

test_image_input = np.expand_dims(test_image, 0)

prediction = model.predict(test_image_input)
predicted_image = np.argmax(prediction, axis=3)
predicted_image = predicted_image[0,:,:]

In [ ]:
plt.figure(figsize=(14,8))
plt.subplot(231)
plt.title("Original Image")
plt.imshow(test_image)
plt.subplot(232)
plt.title("Original Masked image")
plt.imshow(ground_truth_image)
plt.subplot(233)
plt.title("Predicted Image")
plt.imshow(predicted_image)

So the above result is what we get after training our dataset on the Unet model

Now we will explore other models to see if we are able to get better accuracies and results

### Unet with ResNet34

Here we will train our dataset on the Unet model with Resnet34 as it's backbone to see if there are any better results or not

In [ ]:
BACKBONE = 'resnet34'
preprocess_input = sm.get_preprocessing(BACKBONE)
model = sm.Unet(BACKBONE, encoder_weights='imagenet', classes=classes)
model.compile('Adam', loss=sm.losses.bce_jaccard_loss, metrics=metrics)

model.summary()

In [ ]:
history_resnet34_Unet = model.fit(x=X_train,y=y_train,batch_size=16,epochs=100,verbose=1,validation_data=(X_test, y_test),shuffle=False)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize = (18, 5))

loss = loss =  history_resnet34_Unet.history['loss']             
val_loss =  history_resnet34_Unet.history['val_loss']     
epochs = range(1, len(loss) + 1)
axs[0].plot(epochs, loss, 'y', label="Training Loss")
axs[0].plot(epochs, val_loss, 'r', label="Validation Loss")
axs[0].set_title(f"Resnet_Training Vs Validation Loss for")
axs[0].set_xlabel("Epochs")
axs[0].set_ylabel("Loss")
axs[0].legend()

jaccard_coef =  history_resnet34_Unet.history['jaccard_coef']               
val_jaccard_coef =  history_resnet34_Unet.history['val_jaccard_coef']       
epochs = range(1, len(jaccard_coef) + 1)

axs[1].plot(epochs, jaccard_coef, 'y', label="Training IoU")
axs[1].plot(epochs, val_jaccard_coef, 'r', label="Validation IoU")
axs[1].set_title(f"Resnet_Training Vs Validation IoU")
axs[1].set_xlabel("Epochs")
axs[1].set_ylabel("Loss")
plt.legend()

In [ ]:
fig, axs = plt.subplots(1, 1, figsize = (18, 5))

accuracy =history_resnet34_Unet.history['accuracy']             
val_accuracy = history_resnet34_Unet.history['val_accuracy']     
epochs = range(1, len(loss) + 1)

axs[0].plot(epochs, accuracy, 'y', label="Training accuracy")
axs[0].plot(epochs, val_accuracy, 'r', label="Validation accuracy")
axs[0].set_title("Resnet_Training Vs Validation accuracy")
axs[0].set_xlabel("Epochs")
axs[0].set_ylabel("accuracy")
axs[0].legend()

### Unet with VGG19

Here we will train our dataset on the Unet model with VGG19 as it's backbone to see if there are any better results or not

In [ ]:
BACKBONE = 'vgg19'
preprocess_input = sm.get_preprocessing(BACKBONE)
model = sm.Unet(BACKBONE, encoder_weights='imagenet', classes=total_classes)
model.compile('Adam', loss=sm.losses.bce_jaccard_loss, metrics=metrics)

model.summary()

In [ ]:
history_vgg19_Unet  = model.fit(x=X_train,y=y_train,batch_size=16 ,epochs=100,verbose=1,validation_data=(X_test, y_test),shuffle=False)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize = (18, 5))

loss = history_vgg19_Unet.history['loss']             
val_loss = history_vgg19_Unet.history['val_loss']     
epochs = range(1, len(loss) + 1)

axs[0].plot(epochs, loss, 'y', label="Training Loss")
axs[0].plot(epochs, val_loss, 'r', label="Validation Loss")
axs[0].set_title("Vgg19_Training Vs Validation Loss")
axs[0].set_xlabel("Epochs")
axs[0].set_ylabel("Loss")
axs[0].legend()

jaccard_coef = history_vgg19_Unet.history['jaccard_coef']               
val_jaccard_coef = history_vgg19_Unet.history['val_jaccard_coef']       
epochs = range(1, len(jaccard_coef) + 1)

axs[1].plot(epochs, jaccard_coef, 'y', label="Training IoU")
axs[1].plot(epochs, val_jaccard_coef, 'r', label="Validation IoU")
axs[1].set_title("Vgg19_Training Vs Validation IoU")
axs[1].set_xlabel("Epochs")
axs[1].set_ylabel("Loss")
plt.legend()

In [ ]:
fig, axs = plt.subplots(1, 1, figsize = (18, 5))

accuracy =history_vgg19_Unet.history['accuracy']             
val_accuracy = history_vgg19_Unet.history['val_accuracy']     
epochs = range(1, len(loss) + 1)

axs[0].plot(epochs, accuracy, 'y', label="Training accuracy")
axs[0].plot(epochs, val_accuracy, 'r', label="Validation accuracy")
axs[0].set_title("VGG19_Training Vs Validation accuracy")
axs[0].set_xlabel("Epochs")
axs[0].set_ylabel("accuracy")
axs[0].legend()

### Unet with InceptionV3

Here we will train our dataset on the Unet model with InceptionV3 as it's backbone to see if there are any better results or not

In [ ]:
BACKBONE = 'inceptionv3'
preprocess_input = sm.get_preprocessing(BACKBONE)
model = sm.Unet(BACKBONE, encoder_weights='imagenet', classes=total_classes)
model.compile('Adam', loss=sm.losses.bce_jaccard_loss, metrics=metrics)

model.summary()

In [ ]:
history_inceptionv3_Unet  = model.fit(x=X_train,y=y_train,batch_size=16,epochs=100,verbose=1,validation_data=(X_test, y_test),shuffle=False)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize = (18, 5))

loss = history_inceptionv3_Unet.history['loss']            
val_loss = history_inceptionv3_Unet.history['val_loss']    
epochs = range(1, len(loss) + 1)

axs[0].plot(epochs, loss, 'y', label="Training Loss")
axs[0].plot(epochs, val_loss, 'r', label="Validation Loss")
axs[0].set_title("InceptionV3_Training Vs Validation Loss")
axs[0].set_xlabel("Epochs")
axs[0].set_ylabel("Loss")
axs[0].legend()

jaccard_coef = history_inceptionv3_Unet.history['jaccard_coef']               
val_jaccard_coef = history_inceptionv3_Unet.history['val_jaccard_coef']       
epochs = range(1, len(jaccard_coef) + 1)

axs[1].plot(epochs, jaccard_coef, 'y', label="Training IoU")
axs[1].plot(epochs, val_jaccard_coef, 'r', label="Validation IoU")
axs[1].set_title("InceptionV3_Training Vs Validation IoU")
axs[1].set_xlabel("Epochs")
axs[1].set_ylabel("Loss")
plt.legend()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize = (18, 5))

accuracy =history_inceptionv3_Unet.history['accuracy']             
val_accuracy = history_inceptionv3_Unet.history['val_accuracy']     
epochs = range(1, len(loss) + 1)

axs[0].plot(epochs, accuracy, 'y', label="Training accuracy")
axs[0].plot(epochs, val_accuracy, 'r', label="Validation accuracy")
axs[0].set_title("InceptionV3_Training Vs Validation accuracy")
axs[0].set_xlabel("Epochs")
axs[0].set_ylabel("accuracy")
axs[0].legend()